In [11]:
import chess
import chess.engine
import chess.pgn
import random
import numpy as np

In [12]:
def random_board(max_depth=200):
    board = chess.Board()
    depth = random.randrange(0,max_depth)
    
    for i in range(depth):
        all_moves = list(board.legal_moves)
        random_move = random.choice(all_moves)
        board.push(random_move)
        if board.is_game_over():
            break
    return board
def stockfish(board,depth):
    with chess.engine.SimpleEngine.popen_uci('./Content/stockfish_15_x64_avx2.exe') as sf:
        result = sf.analyse(board,chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score

In [13]:
board = random_board()
print(stockfish(board,1))

-1904


In [14]:
class State(object):
    def __init__(self,board=None):
        if board is None:
            self.board = chess.Board()
        else:
            self.board = board
    
    def serialize(self):
        count = 0
        assert self.board.is_valid()
        bstate = np.zeros(64,np.uint8)
        for i in range(64):
            piece = self.board.piece_at(i)
            if piece is not None:
                count += 1
                bstate[i] = {"P":1, "N":2,"B":3,"R":4,"Q":5,"K":6,
                            "p":9, "n":10,"b":11,"r":12,"q":13,"k":14}[piece.symbol()]
            #                 White pawn:1 
            #                 White knight:2
            #                 White Bishop:3
            #                 White rook:4
            #                 White queen:5
            #                 White King:6
            #                 Black pawn:9
            #                 Black knight:10
            #                 Black bishop:11
            #                 Black rook:12
            #                 Black queen:13
            #                 Black king:14

#         print(bstate[0:15])
#         print(bstate[16:47])
#         print(bstate[48:])
#        print(bstate) Encode chess pieces into a 1 dimensional array
#       return state        
        if self.board.has_queenside_castling_rights(chess.WHITE):
            assert bstate[0] == 4
            bstate[0] = 7
        if self.board.has_kingside_castling_rights(chess.WHITE):
            assert bstate[7] == 4
            bstate[7] = 7
        if self.board.has_queenside_castling_rights(chess.BLACK):
            assert bstate[56] == 8+4
            bstate[56] = 8+7
        if self.board.has_kingside_castling_rights(chess.BLACK):
            assert bstate[63] == 8+4
            bstate[63] = 8+7
        
            
        if self.board.ep_square is not None:
            assert bstate[self.board.ep_square] == 0
            bstate[self.board.ep_square] = 8
        bstate = bstate.reshape(8,8)
        state = np.zeros((5,8,8),np.uint8)
        state[0] = (bstate>>3)&1
        state[1] = (bstate>>2)&1
        state[2] = (bstate>>1)&1
        state[3] = (bstate>>0)&1
        state[4] = self.board.turn*1.0 
        
        return state
    
    def edges(self):
        return list(self.board.legal_moves)
    
    def value(self):
        return 1
    

In [15]:
dataset = open("LichessDataSet/lichess_db_standard_rated_2014-07.pgn")
game = chess.pgn.read_game(dataset)
board = game.board()
for j,move in enumerate(game.mainline_moves()):
    board.push(move)
    data = State(board).serialize()
    print(j)
    print(data)
    print('----------------------------------------')

0
[[[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [1 1 1 1 1 1 1 1]]

 [[1 0 0 1 1 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 1 1 0 0 1]]

 [[1 1 1 0 1 1 1 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 0 1 1 1 1]]

 [[1 0 1 1 0 1 0 1]
  [1 1 1 0 1 1 1 1]
  [0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [1 0 1 1 0 1 0 1]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]]
----------------------------------------
1
[[[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]
  [0 0 0 1 0 0 0 0]
  [1 1 1 0 1 1 1 1]
  [1 

In [16]:
def get_dataset(NumSamples):
    DataSet = open("LichessDataSet/lichess_db_standard_rated_2014-07.pgn")
    GN = 1
    X,Y=[],[]
    while GN<NumSamples+1:
        print("parsing game ", GN, "Got ",len(X),"samples")
        GN += 1
        try:
            game = chess.pgn.read_game(DataSet)
        except Exception:
            break
        board = game.board()
        value = {'1/2-1/2':0,'1-0':1,'0-1':-1}[game.headers['Result']]
        for j,move in enumerate(game.mainline_moves()):
            board.push(move)
            data = State(board).serialize()
            X.append(data)
            Y.append(value)
    return X,Y

In [17]:
X,Y = get_dataset(1000)


parsing game  1 Got  0 samples
parsing game  2 Got  90 samples
parsing game  3 Got  161 samples
parsing game  4 Got  245 samples
parsing game  5 Got  300 samples
parsing game  6 Got  353 samples
parsing game  7 Got  412 samples
parsing game  8 Got  435 samples
parsing game  9 Got  530 samples
parsing game  10 Got  553 samples
parsing game  11 Got  627 samples
parsing game  12 Got  702 samples
parsing game  13 Got  817 samples
parsing game  14 Got  914 samples
parsing game  15 Got  1036 samples
parsing game  16 Got  1096 samples
parsing game  17 Got  1193 samples
parsing game  18 Got  1283 samples
parsing game  19 Got  1382 samples
parsing game  20 Got  1449 samples
parsing game  21 Got  1514 samples
parsing game  22 Got  1586 samples
parsing game  23 Got  1641 samples
parsing game  24 Got  1699 samples
parsing game  25 Got  1746 samples
parsing game  26 Got  1810 samples
parsing game  27 Got  1837 samples
parsing game  28 Got  1921 samples
parsing game  29 Got  1995 samples
parsing gam

parsing game  231 Got  16002 samples
parsing game  232 Got  16030 samples
parsing game  233 Got  16103 samples
parsing game  234 Got  16105 samples
parsing game  235 Got  16107 samples
parsing game  236 Got  16146 samples
parsing game  237 Got  16188 samples
parsing game  238 Got  16209 samples
parsing game  239 Got  16329 samples
parsing game  240 Got  16370 samples
parsing game  241 Got  16459 samples
parsing game  242 Got  16474 samples
parsing game  243 Got  16531 samples
parsing game  244 Got  16629 samples
parsing game  245 Got  16749 samples
parsing game  246 Got  16793 samples
parsing game  247 Got  16854 samples
parsing game  248 Got  16913 samples
parsing game  249 Got  16943 samples
parsing game  250 Got  16988 samples
parsing game  251 Got  17001 samples
parsing game  252 Got  17013 samples
parsing game  253 Got  17087 samples
parsing game  254 Got  17232 samples
parsing game  255 Got  17370 samples
parsing game  256 Got  17464 samples
parsing game  257 Got  17580 samples
p

parsing game  453 Got  30102 samples
parsing game  454 Got  30161 samples
parsing game  455 Got  30215 samples
parsing game  456 Got  30250 samples
parsing game  457 Got  30291 samples
parsing game  458 Got  30360 samples
parsing game  459 Got  30456 samples
parsing game  460 Got  30489 samples
parsing game  461 Got  30537 samples
parsing game  462 Got  30659 samples
parsing game  463 Got  30723 samples
parsing game  464 Got  30845 samples
parsing game  465 Got  30883 samples
parsing game  466 Got  30966 samples
parsing game  467 Got  31022 samples
parsing game  468 Got  31105 samples
parsing game  469 Got  31160 samples
parsing game  470 Got  31210 samples
parsing game  471 Got  31271 samples
parsing game  472 Got  31333 samples
parsing game  473 Got  31447 samples
parsing game  474 Got  31484 samples
parsing game  475 Got  31522 samples
parsing game  476 Got  31618 samples
parsing game  477 Got  31681 samples
parsing game  478 Got  31732 samples
parsing game  479 Got  31732 samples
p

parsing game  727 Got  47081 samples
parsing game  728 Got  47181 samples
parsing game  729 Got  47223 samples
parsing game  730 Got  47270 samples
parsing game  731 Got  47334 samples
parsing game  732 Got  47441 samples
parsing game  733 Got  47517 samples
parsing game  734 Got  47534 samples
parsing game  735 Got  47595 samples
parsing game  736 Got  47679 samples
parsing game  737 Got  47824 samples
parsing game  738 Got  47871 samples
parsing game  739 Got  47896 samples
parsing game  740 Got  47997 samples
parsing game  741 Got  48064 samples
parsing game  742 Got  48138 samples
parsing game  743 Got  48190 samples
parsing game  744 Got  48253 samples
parsing game  745 Got  48349 samples
parsing game  746 Got  48413 samples
parsing game  747 Got  48498 samples
parsing game  748 Got  48542 samples
parsing game  749 Got  48553 samples
parsing game  750 Got  48681 samples
parsing game  751 Got  48755 samples
parsing game  752 Got  48812 samples
parsing game  753 Got  48904 samples
p

parsing game  950 Got  61943 samples
parsing game  951 Got  62019 samples
parsing game  952 Got  62091 samples
parsing game  953 Got  62140 samples
parsing game  954 Got  62221 samples
parsing game  955 Got  62299 samples
parsing game  956 Got  62374 samples
parsing game  957 Got  62446 samples
parsing game  958 Got  62500 samples
parsing game  959 Got  62553 samples
parsing game  960 Got  62612 samples
parsing game  961 Got  62659 samples
parsing game  962 Got  62754 samples
parsing game  963 Got  62811 samples
parsing game  964 Got  62853 samples
parsing game  965 Got  62912 samples
parsing game  966 Got  62973 samples
parsing game  967 Got  63082 samples
parsing game  968 Got  63190 samples
parsing game  969 Got  63250 samples
parsing game  970 Got  63323 samples
parsing game  971 Got  63376 samples
parsing game  972 Got  63391 samples
parsing game  973 Got  63470 samples
parsing game  974 Got  63569 samples
parsing game  975 Got  63600 samples
parsing game  976 Got  63679 samples
p

In [20]:
np.savez("dataset.npz",X,Y)